In [213]:
import numpy as np
import pandas as pd

In [214]:
df = pd.read_csv("play_tennis.csv")
df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [215]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in df.columns:
    df[i]=le.fit_transform(df[i])

In [216]:
#selecting only the attribute columns
X = df.iloc[:,1:5].values
#selecting only the class column
y = df.iloc[:,-1].values


array([[2, 1, 0, 1],
       [2, 1, 0, 0]])

In [217]:
len(X)

14

In [6]:
X

array([[2, 1, 0, 1],
       [2, 1, 0, 0],
       [0, 1, 0, 1],
       [1, 2, 0, 1],
       [1, 0, 1, 1],
       [1, 0, 1, 0],
       [0, 0, 1, 0],
       [2, 2, 0, 1],
       [2, 0, 1, 1],
       [1, 2, 1, 1],
       [2, 2, 1, 0],
       [0, 2, 0, 0],
       [0, 1, 1, 1],
       [1, 2, 0, 0]])

In [114]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)
        print(best_feature,best_thresh)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        print(left_idxs,right_idxs,depth)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        



In [115]:
dt = DecisionTree()

In [116]:
dt.fit(X,y)

0 0
[ 2  6 11 12] [ 0  1  3  4  5  7  8  9 10 13] 0
2 0
[0 1 2 5 9] [3 4 6 7 8] 1
0 1
[2 4] [0 1 3] 2
3 0
[1] [0] 3
3 0
[1 4] [0 2 3] 2
1 0
[0] [1] 3


In [105]:
dt.predict(X[:3])

array([0, 0, 1])

In [227]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
       # print('samples',n_samples,n_feats,self.n_features)
        n_labels = len(np.unique(y))
        #print('n_labels', n_labels)
       # print(X)
        #print(y)
        
        # check the stopping criteria
        if (depth>=self.max_depth or n_labels<=1 or n_samples<self.min_samples_split):
            #leaf_value = self._most_common_label(y)
            #print(leaf_value,'***************')
            return  Node(value = np.bincount(y).argmax())#Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        #print('feat_idxs',feat_idxs)
        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)
        
        print(best_feature,best_thresh)
        
        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        print(left_idxs,right_idxs,depth)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def gini_index(self, y):
        n_samples = len(y)
        counts = np.unique(y, return_counts=True)[1]
        impurity = 1
        for count in counts:
            prob = count / n_samples
            impurity -= prob ** 2
        return impurity 

    def _best_split(self, X, y, feat_idxs):
        best_gain = 1
        split_idx, split_threshold = None, None
        if len(X) <= 2:
            for feat_idx in range(0,4):
               # print(feat_idx)
                X_column = X[:, feat_idx]
                #print(X_column)
                thresholds = np.unique(X_column)
                if len(thresholds)==2:
                    split_idx=feat_idx
                    thresh=thresholds[0]
            return split_idx,thresh

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            #if len(thresholds) < 2:
               # continue
            #thresholds= (thresholds[:-1] + thresholds[1:]) / 2

            for thr in thresholds:
                # calculate the information gain
                gain = self.impurity_gain(y, X_column, thr)
               # print(gain)

                if gain!=0 and gain < best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    

    def impurity_gain(self, y, X_column, threshold):
        
        # create children
        left_subset, right_subset = self._split(X_column, threshold)

        if len(left_subset) == 0 or len(right_subset) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_left, n_right = len(left_subset), len(right_subset)
        gini_l, gini_r = self.gini_index(y[left_subset]), self.gini_index(y[right_subset])
        
        return (n_left/n)*gini_l + (n_right/n)*gini_r
        
    
    def _split(self, X, split_thresh):
        left_idxs = np.argwhere(X <= split_thresh).flatten()
        right_idxs = np.argwhere(X > split_thresh).flatten()
        return left_idxs, right_idxs


    #def _most_common_label(self, y):
        #counter = Counter(y)
        #value = counter.most_common(1)[0][0]
        #return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        



In [228]:
dt = DecisionTree()

In [229]:
dt.fit(X,y)

0 0
[ 2  6 11 12] [ 0  1  3  4  5  7  8  9 10 13] 0
2 0
[0 1 2 5 9] [3 4 6 7 8] 1
0 1
[2 4] [0 1 3] 2
3 0
[1] [0] 3
3 0
[1 4] [0 2 3] 2
1 0
[0] [1] 3


In [230]:
dt.predict(X)

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0], dtype=int64)

In [236]:
a=Node()

In [237]:
a.feature

In [192]:
len(X)

2

In [182]:
X[:,0]

array([2, 2])

In [160]:
y =[1 ,0]

In [203]:
X_column = X[:, 3]
                #print(X_column)
thresholds = np.unique(X_column)

In [205]:
thresholds[0]

0

In [208]:
def _best_split(X, y):
        best_gain = 0.5
        split_idx, split_threshold = None, None
        if len(X) <= 2:
            for feat_idx in range(0,4):
                print(feat_idx)
                X_column = X[:, feat_idx]
                #print(X_column)
                thresholds = np.unique(X_column)
                if len(thresholds)==2:
                    threshold=thresholds[0]
        return feat_idx,threshold
            
        for feat_idx in range(0,4):
            print(feat_idx)
            X_column = X[:, feat_idx]
            print(X_column)
            thresholds = np.unique(X_column)
            #if len(thresholds) < 2:
               # continue
            #thresholds= (thresholds[:-1] + thresholds[1:]) / 2

            for thr in thresholds:
                # calculate the information gain
                gain = impurity_gain(y, X_column, thr)
                print(gain)

                if gain!=0 and gain < best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

In [209]:
_best_split(X,y)

0
1
2
3


(3, 0)

In [162]:
def impurity_gain(y, X_column, threshold):
        
        # create children
        left_subset, right_subset = _split(X_column, threshold)

        if len(left_subset) == 0 or len(right_subset) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_left, n_right = len(left_subset), len(right_subset)
        gini_l, gini_r = gini_index(y[left_subset]), gini_index(y[right_subset])
        
        return (n_left/n)*gini_l + (n_right/n)*gini_r
        
    
def _split(X, split_thresh):
    left_idxs = np.argwhere(X <= split_thresh).flatten()
    right_idxs = np.argwhere(X > split_thresh).flatten()
    return left_idxs, right_idxs

In [69]:
np.unique(X[[1,2,3,4,5]][0], return_counts=True)

(array([0, 1, 2]), array([2, 1, 1], dtype=int64))

In [187]:
def gini_index(data):
    n_samples = len(data)
    counts = np.unique(data, return_counts=True)[1]
    impurity = 1
    for count in counts:
        prob = count / n_samples
        impurity -= prob ** 2
    return impurity

In [58]:
gini_impurity(X[3])

0.625

In [56]:
gini_index(X['wind'])

0.48979591836734704

In [40]:
X['outlook'].value_counts()

2    5
1    5
0    4
Name: outlook, dtype: int64

In [74]:
X = df.iloc[:,1:5].values

In [89]:
y[[0,1,2,3,4,5]]

array([0, 0, 1, 1, 1, 0])

In [106]:
def gini_index(y):
    n_samples = len(y)
    counts = np.unique(y, return_counts=True)[1]
    impurity = 1
    for count in counts:
        prob = count / n_samples
        impurity -= prob ** 2
    return impurity 


In [107]:
gini_index(y)

0.4591836734693877

In [108]:
y = df.iloc[:,-1]

In [109]:
gini_index(y)

0.4591836734693877